In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using LCPSim
using Blink
using LearningMPC
using LearningMPC.Models
using RigidBodySim

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [43]:
boxatlas = BoxAtlas();

mvis = MechanismVisualizer(boxatlas)
open(mvis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 7606`, ProcessRunning), TCPSocket(RawFD(68) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(3, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(73) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 3, Nullable{Any}(true))))

In [55]:
lqrsol = LQRSolution(boxatlas)
mpc_params = MPCParams(boxatlas)
# push!(mpc_params.mip_solver.options, (:TimeLimit, 1))
mpc_params.horizon = 1

lqr_mpc_controller = MPCController(boxatlas, mpc_params, lqrsol, Function[lqrsol])

mixed_control, lowlevel_controllers = LearningMPC.build_mixed_controller(mechanism(boxatlas), lqrsol,
    vcat(
        [
            (Point3D(default_frame(findbody(mechanism(boxatlas), bodyname)), 0., 0, 0),
             HalfSpace3D(Point3D(root_frame(mechanism(boxatlas)), 0., 0., 0),
                         FreeVector3D(root_frame(mechanism(boxatlas)), 0., 0, 1)))
            for bodyname in ["r_foot_sole", "l_foot_sole"]
        ],
#         [
#             (Point3D(default_frame(findbody(mechanism(boxatlas), "l_hand_mount")), 0., 0, 0),
#              HalfSpace3D(Point3D(root_frame(mechanism(boxatlas)), 0., 1., 0),
#                          FreeVector3D(root_frame(mechanism(boxatlas)), 0., -1, 0)))
#         ]
    ));

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
contact_state = (true, true)
contact_state = (false, true)
contact_state = (true, false)
contact_state = (false, false)


In [56]:
state = nominal_state(boxatlas)
set_velocity!(state, findjoint(mechanism(boxatlas), "floating_base"), [-0., 0, 0])
mixed_control(zeros(num_velocities(state)), 0.0, state)

11-element Array{Float64,1}:
  -6.11309e-16
  -1.39614e-6 
  -1.38471e-17
  10.9923     
  10.9923     
   6.82181e-11
   7.98648e-11
 -73.8047     
 -73.8047     
 858.761      
 858.761      

In [57]:
state = nominal_state(boxatlas)
set_velocity!(state, findjoint(mechanism(boxatlas), "floating_base"), [-0.3, 0, 0])
problem = LearningMPC.simulation_problem(state, mixed_control, 0.01, 5.0)
@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
setanimation!(mvis, sol)

 45.298000 seconds (842.75 k allocations: 43.873 MiB)


true